In [1]:
import re 
import tweepy
from tweepy import OAuthHandler 
from textblob import TextBlob 

In [2]:
class TwitterClient(object): 
    ''' 
    Generic Twitter Class for sentiment analysis. 
    '''
    def __init__(self): 
        ''' 
        Class constructor or initialization method. 
        '''
        # keys and tokens from the Twitter Dev Console 
        consumer_key = '#insert_your_key'
        consumer_secret = '#insert_your_secret'
        access_token = '#insert_your_token'
        access_token_secret = '#insert_your_token_secret'
  
        # attempt authentication 
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 
  
    def clean_tweet(self, tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()) 
  
    def get_tweet_sentiment(self, tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
  
    def get_tweets(self, query, count, geocode): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count, geocode = geocode) 
            
#             print(fetched_tweets)
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
  
                # saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
  
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return tweets 
            
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e))
        

In [8]:
def main(): 
    # creating object of TwitterClient Class 
    api = TwitterClient() 
    # calling function to get tweets 
    tweets = api.get_tweets(query = 'California, Fire', count = 1000, geocode = "37.801239,-122.258301,30mi")
  
    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # percentage of positive tweets 
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    # percentage of negative tweets 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
    
    # picking neutral tweets from tweets
    neutweets = [tweet for tweet in tweets if tweet['sentiment'] == 'neutral']
    # percentage of neutral tweets 
    print("Neutral tweets percentage: {} %".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))
  
    # printing first 5 positive tweets 
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:1000]: 
        print(tweet['text']) 
  
    # printing first 5 negative tweets 
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:1000]: 
        print(tweet['text'])
        
    # printing first 5 negative tweets 
    print("\n\nNeutral tweets:") 
    for tweet in neutweets[:1000]: 
        print(tweet['text']) 

In [9]:
if __name__ == "__main__": 
    # calling main function 
    main() 

Positive tweets percentage: 21.176470588235293 %
Negative tweets percentage: 23.529411764705884 %
Neutral tweets percentage: 55.294117647058826 %


Positive tweets:
RT @SFUrbanForester: Lorelai Clark of Lone Tree Elementary at Beale AFB, California was the 5th Grade Winner of the 2019 California Arbor W…
Lorelai Clark of Lone Tree Elementary at Beale AFB, California was the 5th Grade Winner of the 2019 California Arbo… https://t.co/0JfQpq5mgs
Miguel Gomez of Valley View Elementary School in Nuevo, California was the 3rd Grade Winner of the 2019 California… https://t.co/zCGVkQfdXw
@EmojiWeatherCA I love this concept but can we find something other than fire to indicate things that aren’t “actua… https://t.co/M1qZsMY6Pf
RT @SFUrbanForester: Congratulations to Ryan Stanly of Mission Hills High School in San Marcos, California for winning the 2019 California…
Congratulations to Ryan Stanly of Mission Hills High School in San Marcos, California for winning the 2019 Californ… https://t.co/61